In [1]:
%load_ext autoreload
%autoreload 2

In [87]:
#manipulação e modelagem
import pandas as pd
import numpy as np
import lightgbm as lg

#gráficos e visualizações
import matplotlib.pyplot as plt
import seaborn as sns

#scrapping
import yfinance as yf

from utils import construir_base





Construindo a base para modelarmos - em utils ler construir_base <br>
<br>
1. trazer informações utilizando yahoo API<br>
2. Preenche dados faltantes de quando a bolsa nao opera<br>
3. feat eng.<br>
   3.1 features temporais - dia da semana, dia do ano, comeco/final  de mês etc <br>
   3.2 features temporal sazonal - modelar sazonalidade semanal, mensal e anual utilizando senoides <br>
   3.3 features autoregressivas - médias móveis, deltas etc <br>

In [117]:
historico, quarterly_income_stmt, quartly_balance_sheet,quarterly_cashflow = construir_base()

c:\Users\fuksy\OneDrive\Desktop\github\stockmarket_prediction\notebook\utils.py:106: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  historico['Low_delta_ma_'+str(i)] = historico['Low_ma_'+str(i)] - historico['Low']
c:\Users\fuksy\OneDrive\Desktop\github\stockmarket_prediction\notebook\utils.py:107: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  historico['Close_delta_ma_'+str(i)] = historico['Close_ma_'+str(i)] - historico['Close']
c:\Users\fuksy\OneDrive\Desktop\github\stockmarket_prediction\notebook\utils.py:108: PerformanceWarn

In [118]:
historico.to_csv('historico.csv',sep = ',')

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,bolsa_opera,dow,...,Open_ma_150,High_ma_150,Low_ma_150,Close_ma_150,Volume_ma_150,Open_delta_ma_150,High_delta_ma_150,Low_delta_ma_150,Close_delta_ma_150,Volume_delta_ma_150
0,2018-09-14,6.696023,6.753010,6.582048,6.692461,55756300.0,0.0,0.0,1.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-09-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-09-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-09-17,6.635474,6.941782,6.635474,6.909727,71289500.0,0.0,0.0,1.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-09-18,6.945345,7.212473,6.934659,7.212473,72959100.0,0.0,0.0,1.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2023-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,...,26.659269,27.057113,26.322227,26.731606,5.926055e+07,NaN,NaN,NaN,NaN,NaN
1823,2023-09-11,33.470001,33.689999,33.330002,33.369999,30276200.0,0.0,0.0,1.0,0,...,26.717722,27.112092,26.379390,26.785571,5.889557e+07,-6.752279,-6.577906,-6.950612,-6.584428,2.861937e+07
1824,2023-09-12,33.549999,33.779999,33.490002,33.529999,31963000.0,0.0,0.0,1.0,1,...,26.771110,27.173418,26.435550,26.848883,5.905122e+07,-6.778890,-6.606581,-7.054452,-6.681116,2.708822e+07
1825,2023-09-13,33.650002,33.700001,33.029999,33.029999,47685700.0,0.0,0.0,1.0,2,...,26.833766,27.236528,26.500164,26.912417,5.903984e+07,-6.816236,-6.463472,-6.529835,-6.117581,1.135414e+07


In [95]:
teste = pd.Series([np.nan, 10, 5, 6, np.nan, 30, 10,10,10,5, 10, 8])
np.mean(teste)

10.4

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2018-09-14,6.696024,6.753011,6.582049,6.692463,55756300,0.0,0.0
1,2018-09-17,6.635474,6.941782,6.635474,6.909727,71289500,0.0,0.0
2,2018-09-18,6.945343,7.212471,6.934658,7.212471,72959100,0.0,0.0
3,2018-09-19,7.098498,7.287269,7.062881,7.116307,60563700,0.0,0.0
4,2018-09-20,7.216033,7.248088,7.034385,7.077126,56206900,0.0,0.0
...,...,...,...,...,...,...,...,...
1235,2023-09-08,33.480000,33.669998,33.259998,33.400002,39074100,0.0,0.0
1236,2023-09-11,33.470001,33.689999,33.330002,33.369999,30276200,0.0,0.0
1237,2023-09-12,33.549999,33.779999,33.490002,33.529999,31963000,0.0,0.0
1238,2023-09-13,33.650002,33.700001,33.029999,33.029999,47685700,0.0,0.0


In [25]:
acao.cashflow

,2022-12-31,2021-12-31,2020-12-31,2019-12-31
Free Cash Flow,40136000000.0,31466000000.0,23016000000.0,1703000000.0
Repayment Of Debt,-14764000000.0,-27240000000.0,-31607000000.0,-32480000000.0
Issuance Of Debt,2880000000.0,1885000000.0,17023000000.0,7464000000.0
Capital Expenditure,-9581000000.0,-6325000000.0,-5874000000.0,-23897000000.0
End Cash Position,7996000000.0,10480000000.0,11725000000.0,7377000000.0
...,...,...,...,...
Operating Gains Losses,4390000000.0,9386000000.0,10296000000.0,4381000000.0
Pension And Employee Benefit Expense,1228000000.0,2098000000.0,-1001000000.0,2086000000.0
Earnings Losses From Equity Investments,-251000000.0,-1607000000.0,659000000.0,-153000000.0
Net Foreign Currency Exchange Gain Loss,4557000000.0,10795000000.0,11094000000.0,8460000000.0
